In [1]:
import pandas as pd
import numpy as np
import re
from better_profanity import profanity
import en_core_web_sm
nlp = en_core_web_sm.load()

In [2]:
origin = pd.read_csv('output/corpus/tldr_20_200_100_600.csv')
origin.head()

,author,content,summary,content_len,summary_len,id,subreddit,subreddit_id,title
0,iamacannibal,Theres an entire small town under the lake by ...,I'll try and get some similar shots from lake ...,181,25,c6aveyw,AbandonedPorn,t5_2sh6t,NaN
1,leep420,I take a beta blocker for my heart condition t...,Butchered daughter to save the universe and wa...,219,27,c6c7tdy,AskReddit,t5_2qh1i,NaN
2,thegoodweretaken,I was hanging out with friends when suddenly t...,I dreamed about being in a white room where I ...,153,33,c6c8fu1,AskReddit,t5_2qh1i,NaN
3,canamrock,While I don't much support the term noob per s...,A 'noob' is someone who acts without a plan an...,515,41,c6d1e2y,tf2,t5_2qka0,NaN
4,mtd14,Analysts are actually surprisingly correct on ...,"Analysts usually do quite well, but Apple not ...",145,31,c6d1irc,apple,t5_2qh1f,NaN


In [3]:
len(origin)

1341180

In [5]:
import contractions
from bs4 import BeautifulSoup
from html import unescape

def clean_text(text):
    cleanr = re.compile('<.*?>') ## clean tags
    text = re.sub(cleanr, '', text)
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) ## remove URL
    text = BeautifulSoup(unescape(text), 'lxml').text ## remove strange characters like &amp; 
    text = contractions.fix(text)
    text = re.sub(r'[_"\-%()|+&=*%#@\[\]/\r\n\t]', '', text) ## remove other unwanted characters
    one = re.sub(r"([eoEO])\1\1+",r"\1\1",text) ## if more than 2 consecutive e or o, remove them until there's only two
    text = re.sub(r"([^eoEO])\1\1+",r"\1",one) ## if more than 2 consecutive other characters, remove them until there's only one
    text = profanity.censor(text) ## remove badwords
    #text = re.sub(r"\d+", "#", s) ## remove numbers
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    text = re.sub(r"([!$*.,?:])\1\1+",r"\1",text) ## remove consecutive *  
    text = re.sub(r"([,.?!:])",r" \1 ",text) ## add space before punctuations    
    text = re.sub(' +', ' ',text) ## remove redundant spaces
    return text

In [6]:
current = origin.loc[:,['content','summary']]

In [7]:
current['text'] = current['content'].apply(lambda x: clean_text(x))
current['summ'] = current['summary'].apply(lambda x: clean_text(x))

In [8]:
tldr = current.loc[:,['text','summ']]

In [9]:
tldr['text_len'] = tldr['text'].apply(lambda x: len(x.split()))
tldr['summ_len'] = tldr['summ'].apply(lambda x: len(x.split()))

In [10]:
tldr.text_len.describe(include="all").apply(lambda x: format(x, 'f'))   

count    1341180.000000
mean         314.532908
std          146.318968
min            1.000000
25%          193.000000
50%          284.000000
75%          414.000000
max         1397.000000
Name: text_len, dtype: object

In [11]:
tldr.summ_len.describe(include="all").apply(lambda x: format(x, 'f')) 

count    1341180.000000
mean          45.981750
std           29.344522
min            1.000000
25%           28.000000
50%           36.000000
75%           52.000000
max         1175.000000
Name: summ_len, dtype: object

In [12]:
tldr.to_csv("output/corpus/tldr_20_200_100_600_clean.csv", index = False)

In [13]:
filtered1 = tldr[tldr.text_len<512].reset_index(drop=True)
filtered1 = tldr[tldr.text_len>=128].reset_index(drop=True)
filtered1 = tldr[tldr.summ_len>=16].reset_index(drop=True)
filtered1 = tldr[tldr.summ_len<64].reset_index(drop=True)

In [14]:
len(filtered1)

1117526

In [15]:
filtered1

,text,summ,text_len,summ_len
0,there is an entire small town under the lake b...,I will try and get some similar shots from lak...,203,28
1,I take a beta blocker for my heart condition t...,Butchered daughter to save the universe and wa...,238,29
2,I was hanging out with friends when suddenly t...,I dreamed about being in a white room where I ...,168,36
3,While I do not much support the term noob per ...,A 'noob' is someone who acts without a plan an...,575,43
4,Analysts are actually surprisingly correct on ...,"Analysts usually do quite well , but Apple not...",159,34
5,Like I said I do not know and do not have the ...,I get you think that through some fancy handwa...,252,44
6,she is been honest with you the entire time . ...,create your own natural highs . Gym get a phys...,210,29
7,If this Plan B is the same as the morning afte...,is I am all for it being somewhat more easily ...,132,58
8,Guess I will throw my . 02 in here . SpecSpect...,Spectral Blades Deep Cuts still procs CM from ...,454,50
9,"to make matters worse I live in new zealand , ...",There is a USB mini connector on the rear of y...,209,30


In [16]:
filtered1.to_csv('output/sub-set/tldr_clean_large.csv',index=False)

In [2]:
import pandas as pd
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [3]:
df = pd.read_csv('output/sub-set/tldr_clean.csv')

In [5]:
lines = len(df)
lines

235292

In [23]:
per_batch = int(lines/5)
for i in range(5):
    df2 = df[per_batch*i:per_batch*(i+1)]
    df2.to_csv(r'output/tldr/part'+str(i)+'.csv',index=False)

"I take a beta blocker for my heart condition that not only makes me have crazy dreams , they can feel more real than reality . 2 dreams come to mind which were fucked up . The last and most recent was basically unzipping the skin on my barely 1 year old daughter to save the universe under the direction of an alien . I knew it was fucked up , but I honestly believed what the alien was saying so to save the universe I butchered my daughter . Now that was pretty bad and all , but the first really bad dream I had on the beta blocker has stuck with me just as much . In it I was at my parents house with all of us dressed in * gear . I then became aware that I was in a dream and thought to myself wtf ? a lucid dream ? this is awesome . At that point my feet wouldn't move and my parents gradually walked up to me before violently * me . I couldn't do anything to wake up and it felt as real as anything I have ever done while awake and sober . It felt like the dream lasted hours , I woke up in t

In [21]:
n1=100
print(df.text[n1]+'\n')
ents = nlp(df.text[n1]).ents
for ent in ents:
    print(ent.text, ent.label_)

No . This is a terrible misquote . It seems like it will work similarly to Arcade games . You buy a disc as an alternative to network installation , you install the game and link it to your GT . Since your console is associated with your GT , anyone on that console can play the game . If you log in from a different console , you can play , but others cannot without you . Once you leave , the users on the other console will be prompted to purchase the game . They won't be prompted to buy your copy out from under you without your consent . That is preposterous . Why would anyone ever assume it would work that way ? Look , we don't know the details of game ownership transfers , but Microsoft has confirmed that it will be fee free . Hopefully it will be painless , but I'm going to assume that there will be some sort of stipulations to avoid abuse game sharing . This could be something like a timer you can't transfer ownership until you've owned a game for two weeks , or a no loopback you c

In [8]:
def count_ent(content):
    if count_ent.counter%1000 ==0:
        print('total processed:',count_ent.counter)
    count_ent.counter += 1
    temp_c = 0
    ents = nlp(content).ents
    for ent in ents:
        if ent.label_ in ['ORG', 'PRODUCT', 'FAC','WORK_OF_ART','MONEY']:
            temp_c = temp_c+1
    return temp_c
count_ent.counter=0

In [24]:
###### recognizing entities ##########
import time


#for i in range(5):
#    if i==0:
#        continue
#    start_t = time.time()
#    df = pd.read_csv('output/tldr/part'+str(i)+'.csv')
#    print('Processing batch: '+str(i))
#    df['ent_count'] = df['text'].apply(lambda x : count_ent(x))
#    df.to_csv(r'output/tldr/part'+str(i)+'.csv',index=False)
#    end_t = time.time()
#    print('spent time:',str(end_t-start_t))



Processing batch: 1
total processed: 0
total processed: 1000
total processed: 2000
total processed: 3000
total processed: 4000
total processed: 5000
total processed: 6000
total processed: 7000
total processed: 8000
total processed: 9000
total processed: 10000
total processed: 11000
total processed: 12000
total processed: 13000
total processed: 14000
total processed: 15000
total processed: 16000
total processed: 17000
total processed: 18000
total processed: 19000
total processed: 20000
total processed: 21000
total processed: 22000
total processed: 23000
total processed: 24000
total processed: 25000
total processed: 26000
total processed: 27000
total processed: 28000
total processed: 29000
total processed: 30000
total processed: 31000
total processed: 32000
total processed: 33000
total processed: 34000
total processed: 35000
total processed: 36000
total processed: 37000
total processed: 38000
total processed: 39000
total processed: 40000
total processed: 41000
total processed: 42000
tota

In [9]:
df = pd.read_csv('output/tldr/part0.csv')
#    print('Processing batch: '+str(i))
df['ent_count'] = df['text'].apply(lambda x : count_ent(x))
df.to_csv(r'output/tldr/part0.csv',index=False)
end_t = time.time()
print('spent time:',str(end_t-start_t))


total processed: 0
total processed: 1000
total processed: 2000
total processed: 3000
total processed: 4000
total processed: 5000
total processed: 6000
total processed: 7000
total processed: 8000
total processed: 9000
total processed: 10000
total processed: 11000
total processed: 12000
total processed: 13000
total processed: 14000
total processed: 15000
total processed: 16000
total processed: 17000
total processed: 18000
total processed: 19000
total processed: 20000
total processed: 21000
total processed: 22000
total processed: 23000
total processed: 24000
total processed: 25000
total processed: 26000
total processed: 27000
total processed: 28000
total processed: 29000
total processed: 30000
total processed: 31000
total processed: 32000
total processed: 33000
total processed: 34000
total processed: 35000
total processed: 36000
total processed: 37000
total processed: 38000
total processed: 39000
total processed: 40000
total processed: 41000
total processed: 42000
total processed: 43000
t

NameError: name 'time' is not defined

In [1]:
import pandas as pd
import numpy as np
import re

In [10]:
df = pd.read_csv('output/tldr/part0.csv')
for i in range(1,5):
    df = df.append(pd.read_csv('output/tldr/part'+str(i)+'.csv'))

In [11]:
df

,text,summ,text_len,summ_len,ent_count
0,I take a beta blocker for my heart condition t...,Butchered daughter to save the universe and wa...,235,29,0
1,I was hanging out with friends when suddenly t...,I dreamed about being in a white room where I ...,166,36,0
2,Analysts are actually surprisingly correct on ...,"Analysts usually do quite well , but Apple not...",158,34,5
3,Backstory : This F250 has been in my family si...,Truck caught fire then later found pieces of p...,246,24,1
4,I a guy was talking to my female friend . She ...,I'm sure it's extremely dependent on the girl ...,198,28,0
5,Longterm CM player here . . started out playin...,I ramble my previous experiences with FM and I...,288,31,4
6,"As the saying goes , Good artists create , gre...",There's nothing wrong with a little noncommerc...,189,23,1
7,Maybe the woman just didn't want to see * get ...,Fighting sometimes gets people killed over stu...,318,41,0
8,Highly educated Americans were sold the belief...,It was a scam that targeted highly educated Am...,106,22,2
9,"It's not particularly humane , but . Boat peop...",Australia does a good job permanently resettli...,271,43,1


In [12]:
df2 = df.loc[df.ent_count>0].reset_index(drop=True)
df2

,text,summ,text_len,summ_len,ent_count
0,Analysts are actually surprisingly correct on ...,"Analysts usually do quite well , but Apple not...",158,34,5
1,Backstory : This F250 has been in my family si...,Truck caught fire then later found pieces of p...,246,24,1
2,Longterm CM player here . . started out playin...,I ramble my previous experiences with FM and I...,288,31,4
3,"As the saying goes , Good artists create , gre...",There's nothing wrong with a little noncommerc...,189,23,1
4,Highly educated Americans were sold the belief...,It was a scam that targeted highly educated Am...,106,22,2
5,"It's not particularly humane , but . Boat peop...",Australia does a good job permanently resettli...,271,43,1
6,"This all day long , i was on with my brother l...",People on Smurf accounts or who CLAIM to be on...,147,31,3
7,"Dude i have a smurf as well , and I dislike th...","when i play on my smurf I dont give a * , i tr...",114,42,2
8,"As a PersianAmerican , I understand what you m...",urban Iranians aren't insane terrorists that r...,125,26,2
9,Here's the thing . He definitely made a point ...,"Was his point correct ? Maybe , but it's not n...",224,29,1


In [13]:
df2.text_len.describe(include="all").apply(lambda x: format(x, 'f'))   

count    134742.000000
mean        213.236437
std          61.823387
min          16.000000
25%         161.000000
50%         209.000000
75%         264.000000
max         488.000000
Name: text_len, dtype: object

In [14]:
df2.summ_len.describe(include="all").apply(lambda x: format(x, 'f'))   

count    134742.000000
mean         33.977706
std          10.448021
min           1.000000
25%          26.000000
50%          31.000000
75%          40.000000
max          63.000000
Name: summ_len, dtype: object

In [16]:
df2.ent_count.describe(include="all").apply(lambda x: format(x, 'f'))   

count    134742.000000
mean          2.974069
std           2.696239
min           1.000000
25%           1.000000
50%           2.000000
75%           4.000000
max          43.000000
Name: ent_count, dtype: object

In [17]:
df3 = df2[df2.text_len<=256].reset_index(drop = True)
df3

,text,summ,text_len,summ_len,ent_count
0,Analysts are actually surprisingly correct on ...,"Analysts usually do quite well , but Apple not...",158,34,5
1,Backstory : This F250 has been in my family si...,Truck caught fire then later found pieces of p...,246,24,1
2,"As the saying goes , Good artists create , gre...",There's nothing wrong with a little noncommerc...,189,23,1
3,Highly educated Americans were sold the belief...,It was a scam that targeted highly educated Am...,106,22,2
4,"This all day long , i was on with my brother l...",People on Smurf accounts or who CLAIM to be on...,147,31,3
5,"Dude i have a smurf as well , and I dislike th...","when i play on my smurf I dont give a * , i tr...",114,42,2
6,"As a PersianAmerican , I understand what you m...",urban Iranians aren't insane terrorists that r...,125,26,2
7,Here's the thing . He definitely made a point ...,"Was his point correct ? Maybe , but it's not n...",224,29,1
8,Except that Brony carries with it a connotatio...,"I have had bad experiences with Bronies , and ...",113,24,3
9,It's definately a good step to clean up in hig...,If Riot really wants punishment this harsh for...,249,40,4


In [18]:
df2.to_csv(r'output/tldr/tdlr_ent_filtered.csv',index=False)